In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

def loglog(xs, ys):
    plt.loglog(xs, ys, ls='none', marker='.')


In [ ]:
# Load data
df     = pd.read_csv('data/bbs/bbs2011.csv')
df2    = pd.read_csv('data/bbs/speciesTableBody.csv', index_col='sppKey')
df2['rate'] = df2.mass**0.75
df3 = pd.merge(df, df2, left_on=['spp',], right_index=True)

df3['log_rate']  = np.log(df3.rate)
df3['log_abund'] = np.log(df3.abund)

# Number of different locations
num_routes = df['route'].nunique()

if False:
    # Scatter plot of non-grouped data
    plt.figure()
    p=sns.regplot('log_abund', 'log_rate', data=df3)
    model = sm.OLS(exog=sm.add_constant(df3.log_abund.values), endog=df3.log_rate.values)
    print(model.fit().summary())
    

In [ ]:
# Scatter plot of species-level grouped data

# Find mean abundances for species that appear in at least 10 locations
g = df[['spp','abund']].groupby('spp')
# g = g.filter(lambda x: len(x) > 0).groupby('spp')

counts = g.abund.agg(abund_sum='sum',
                     abund_num=len,
                     abund_p1=lambda x: (x==1).sum(),
                     abund_mean=lambda x: x.sum()/num_routes,
                     abund_var =lambda x: (x**2).sum()/num_routes - (x.sum()/num_routes)**2)

# Plot abundances versus metabolic rates
df4 = pd.merge(counts, df2, left_index=True, right_index=True)
del counts

df4['log_rate']  = np.log(df4.rate)
df4['log_abund_sum'] = np.log(df4.abund_sum)

df4 = df4.sort_values('abund_num', ascending=False)


for cdata in [df4, df4.iloc[:50]]:
    plt.figure()
    p=sns.regplot('log_abund_sum', 'log_rate', data=cdata)
    plt.title('# species = %d' % len(cdata))
    plt.show()

    print("Species-level sum abundance vs. metabolic rate")
    model = sm.OLS(exog=sm.add_constant(cdata.log_abund_sum.values), endog=cdata.log_rate.values)
    print(model.fit().summary())

In [ ]:
# Negative binomial fits for top  species
size_params, p_params = np.array([]), np.array([])

top_species = df4[:100]

for spp, _ in top_species.iterrows():
    samp = df[df.spp == spp].abund.values
    samp = np.append(samp, np.zeros(num_routes-len(samp)))
    res  = sm.NegativeBinomial(samp, np.ones_like(samp)).fit(disp=0)

    size = 1. / res.params[1] 
    prob = size / (size + np.exp(res.params[0]))
    size_params = np.append(size_params, size)
    p_params    = np.append(p_params, prob)


In [ ]:
#plt.scatter(top_species.abund_mean, size_params * (1-p_params) / p_params)
plt.figure(figsize=(14,3))
plt.subplot(1,4,1)
loglog(top_species.abund_mean, size_params * (1-p_params) / p_params)
plt.plot(plt.xlim(), plt.xlim(), '--')
plt.xlabel('Empirical abundance sum')
plt.ylabel('NegativeBinomial sum')

plt.subplot(1,4,2)
loglog(top_species.abund_var, size_params * (1-p_params) / p_params**2)
plt.plot(plt.xlim(), plt.xlim(), '--')
plt.xlabel('Empirical abundance variance')
plt.ylabel('NegativeBinomial variance')

plt.subplot(1,4,3)
# NegativeBinomial probability of zero is given by p**r
plt.plot((1-top_species.abund_num/num_routes), p_params**size_params, ls='none', marker='.')
plt.plot(plt.xlim(), plt.xlim(), '--')
plt.xlabel('Empirical p(0)')
plt.ylabel('NegativeBinomial p(0)')

# NegativeBinomial probability of one is given by r*p**r*(1-p)
plt.subplot(1,4,4)
plt.plot(top_species.abund_p1/num_routes, size_params*p_params**size_params*(1-p_params), ls='none', marker='.')
plt.plot(plt.xlim(), plt.xlim(), '--')
plt.xlabel('Empirical p(1)')
plt.ylabel('NegativeBinomial p(1)')
plt.tight_layout()

plt.figure()
loglog(top_species.abund_sum, size_params)
plt.xlabel('Sum abundance')
plt.ylabel('NegativeBinomial size parameter')




In [ ]:
# Generative model ---
#  Actual abundances are drawn from some distribution (e.g., lognormal)
#  Metabolic rates assigned according to exponential distribution, r ~ exp(-beta*abundance)
#  Location abundances are sampled from NegativeBinomial
#  We then measure correlation between location abudances and metabolic rates

N = 1000
if True:
    log_std    = np.std(np.log(df4.abund_sum.values))
    log_mean   = np.mean(np.log(df4.abund_sum.values))
    abundances = np.exp(np.random.normal(size=(N,))*log_std + log_mean)
else:
    abundances = np.random.logseries(0.999975, size=(N,))
    print(abundances.mean(), df4.abund_sum.mean())
    
if False:
    plt.figure()
    plt.subplot(1,2,1)
    plt.hist(np.log(df4.abund_sum.values), bins=100)
    plt.subplot(1,2,2)
    plt.hist(np.log(abundances), bins=100)   

# Clip abundances to max value, and make sure all abundances are non-zero
abundances = abundances[abundances < df4.abund_sum.max()]
abundances = abundances[abundances > 0]    

# Draw metabolic rates according to exponential distribution
beta = 200
metabolic_rates = np.array([np.random.exponential(beta/x) for x in abundances])

# Consider only species that have some minimal metabolic rate
abundances      = abundances[metabolic_rates >= df4.rate.min()]
metabolic_rates = metabolic_rates[metabolic_rates >= df4.rate.min()]

print('Mean rate', df4.rate.mean() , metabolic_rates.mean())

plt.loglog(abundances, metabolic_rates, ls='none', marker='.')
plt.xlabel('Actual abundance')
plt.ylabel('Metabolic rates')
plt.show()

model = sm.OLS(exog=sm.add_constant(np.log(abundances)), endog=np.log(metabolic_rates))
print(model.fit().summary())

In [ ]:
#r_values = r_func(abundances)
#r_values = 0.1*np.ones_like(abundances)
r_values = np.random.choice(size_params, size=(N,))
#r_values = 0.1*np.exp(np.random.normal(size=(N,)))

def exp_sampler(ndx):
    return np.random.exponential(abundances[ndx])
def poisson_sampler(ndx):    
    return np.random.poisson(abundances[ndx])
def nb_sampler(ndx): # Negative binomial model
    r=r_values[ndx]
    return np.random.negative_binomial(r, r/(abundances[ndx]+r)) 
    
samplefunc          = nb_sampler
#samplefunc          = poisson_sampler
location_abundances = np.array([samplefunc(ndx) for ndx in range(len(abundances))])

endog = np.log(metabolic_rates)[location_abundances > 0]
exog  = np.log(location_abundances)[location_abundances > 0]

model = sm.OLS(exog=sm.add_constant(exog), endog=endog)
print(model.fit().summary())
plt.loglog(location_abundances, metabolic_rates, ls='none', marker='.')
plt.xlabel('Location abundance')
plt.ylabel('Metabolic rates')
plt.show()